In [1]:
import os, sys, subprocess, cupy as cp, platform
print("Kernel Python:", sys.executable)
print("Platform:", platform.system())
print("CUDA_PATH:", os.environ.get("CUDA_PATH"))
print("LD_LIBRARY_PATH:", os.environ.get("LD_LIBRARY_PATH"))
print("nvcc path:", subprocess.check_output(["which","nvcc"]).decode().strip())
print(subprocess.check_output(["nvcc","--version"]).decode().splitlines()[-1])
print("Device count (CuPy):", cp.cuda.runtime.getDeviceCount())


/home/user/miniconda/envs/rapids/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Kernel Python: /home/user/miniconda/envs/rapids/bin/python
Platform: Linux
CUDA_PATH: /usr/local/cuda
LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64:
nvcc path: /home/user/miniconda/envs/rapids/bin/nvcc
Build cuda_12.9.r12.9/compiler.36037853_0
Device count (CuPy): 1


In [ ]:
import cupy as cp
import numpy as np
from cuml.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xarray as xr
import numpy as np
import pandas as pd
import os
import gc

In [3]:
data = pd.read_excel("./Users/user/OneDrive/Desktop/gdal_trials/Final_Data 5.xlsx")

In [4]:
x = data.drop('PM2.5', axis=1)
x = x.drop(['time', 'latitude', 'longitude', 'Abr'], axis=1)
#x = x.drop('Stations', axis=1)
#x = x.drop('longitude_c', axis=1)
#x = x.drop('latitude_c', axis=1)
y = data['PM2.5']

In [5]:
best_params = {'n_estimators': 406,
                'max_depth': 33,
                'min_samples_split': 6,
                'min_samples_leaf': 1,
                'max_features': 0.10350217444283559,
                'bootstrap': False}

In [6]:
model = RandomForestRegressor(**best_params, random_state=42)
model.fit(
        x,
        y
    )

RandomForestRegressor()

In [7]:
folder = './Users/user/OneDrive/Desktop/gdal_trials/dss/'
files = os.listdir(folder)
len(files)

414

In [8]:
np.unique(xr.open_dataset(folder+files[0]).to_dataframe().columns == x.columns)

array([ True])

In [9]:
pred_pm = []
for i in range(len(files)):
    df = xr.open_dataset(folder + files[i]).to_dataframe().reset_index()
    coords_df = df[['time', 'latitude', 'longitude']]
    df.drop(['time', 'latitude', 'longitude'], axis=1, inplace=True)
    preds = pd.Series(index=df.index, data= model.predict(df), name='Pred_PM2.5')
    pred_pm.append(pd.concat([coords_df, preds], axis=1).set_index(['time', 'latitude', 'longitude']))

In [10]:
pd.concat(pred_pm).to_xarray().to_netcdf('./Users/user/OneDrive/Desktop/gdal_trials/Predicted PM2.5/pred_pm_3.nc')